# Imports and setup

In [1]:
# !pip install pgmpy
# !pip install networkx==2.8.8
# !pip install interpret

In [1]:
import pgmpy
import random
import networkx as nx
import pylab as plt
import numpy as np
import pandas as pd
from collections import defaultdict

from decimal import Decimal # For high-precision calculations, like fracturing the probability space
from networks import *

from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import scienceplots
plt.style.use("default")
plt.style.use(["science","ieee"])

# Bayesian network graph

In [2]:
# B, G = BN_lines([3,4])
B, G = BN_forest([3,3])

# Helper functions

In [3]:
def ebm_polynomial(ebm):
    """
    Given GA^2M (EBM) model, return the polynomial giving log-odds
    """
    poly = defaultdict(Decimal)
    const = Decimal(ebm.intercept_[0])
    linear_terms = ebm.term_scores_[:len(G)-1]
    linear_terms = [[Decimal(w) for w in v] for v in linear_terms]
    linear_term_names = [name for name in ebm.term_names_ if "&" not in name]
    inter_terms = ebm.term_scores_[len(G)-1:]
    inter_terms = [[[Decimal(x) for x in w] for w in v] for v in inter_terms]
    inter_term_names = [name.split(" & ") for name in ebm.term_names_ if "&" in name]
    for x_i, term in zip(linear_term_names, linear_terms):
        # f(x) = (term[1] if x==0), (term[2] if x==1)
        # f(x) = (1-x)*(term[1]) + x*(term[2])
        # f(x) = x*(term[2]-term[1]) + term[1]
        poly[f"x_{x_i}"] += term[2]-term[1]
        const += term[1]

    for (x_i, x_j), term in zip(inter_term_names, inter_terms):
        # f(x_i,x_j) = term[1,1] if x_i==0, x_j==0
        #              term[1,2] if x_i==0, x_j==1
        #              term[2,1] if x_i==1, x_j==0
        #              term[2,2] if x_i==1, x_j==1
        # f(x_i,x_j) = x_i*x_j*term[2,2] + x_i*(1-x_j)*term[2,1] + (1-x_i)*x_j*term[1,2] + (1-x_i)*(1-x_j)*term[1,1]
        # f(x_i,x_j) = x_i*x_j*(term[2,2]-term[2,1]-term[1,2]+term[1,1]) + x_i*(term[2,1]-term[1,1]) + x_j*(term[1,2]-term[1,1]) + term[1,1]
        poly[f"x_{x_i}*x_{x_j}"] = term[2][2]-term[2][1]-term[1][2]+term[1][1]
        poly[f"x_{x_i}"] += term[2][1]-term[1][1]
        poly[f"x_{x_j}"] += term[1][2]-term[1][1]
        const += term[1][1]
    poly["const"] = const
    return poly

def predict_poly(poly, vals=[1,1,1,1,1,1,1]):
    out = 0
    for name, coef in poly.items():
        if name == "const":
            out += coef
        elif "*" in name: # Interaction terms
            i, j = name.replace("x_", "").split("*")
            out += vals[int(i)-1]*vals[int(j)-1]*coef
        else: # Linear terms:
            i = int(name.replace("x_", ""))
            out += vals[i-1]*coef
    return out

## Create polynomials for BN and GAM (10 trials)

In [8]:
latex_polynomials_bn = []
latex_polynomials_gam = []

for seed in tqdm(range(10, 20),desc="Seed", colour="purple"):
    np.random.seed(seed)
    y_size = 2**14
    y = np.random.choice([0,1], y_size)
    bn = BN(G, seed=seed, bounds = (0.2, 0.8), mode="extreme")
    X = bn.generate_dataset(y, seed=seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
    
    # Round all to 3dp so it aligns with table in LaTeX file.
    for k in bn.dists.keys():
        newval = np.around(bn.dists[k], 3)
        newval[newval == 0] = 0.001 # Change 0.000 to 0.001.
        bn.dists[k] = newval
        
    # Generate polynomial in LaTeX format
    p = bn.polynomial
    linear = [f"{p[k]:.2f}{k}" for k in p if "*" not in k]
    const = linear[-1]
    del linear[-1]
    linear = sorted(linear, key=lambda x:int(x.split("_")[1])) # Sort by name of x-variable
    linear.insert(0, const) # Move constant to front
    interaction = [f"{p[k]:.2f}{k}" for k in p if "*" in k]
    for i, term in enumerate(linear): # Put curly braces for subscripts
        if "_" in term:
            linear[i] = term.replace("_", "_{") + "}"
    for i, term in enumerate(interaction): # Put curly braces for subscripts
        term = term.replace("*","").split("x_")
        term = term[0] + "x_{" + term[1] + "}x_{" + term[2] + "}"
        interaction[i] = term
    out_str = " + ".join(linear).replace("const", "")
    out_str += " + "
    out_str += " + ".join(interaction).replace("*", "")
    out_str = out_str.replace("+ -", "- ")
    latex_polynomials_bn.append(out_str)
    
    ebm = ExplainableBoostingClassifier(interactions=len(bn.interior_edges), random_state=seed)
    ebm.fit(X_train, y_train) 
    
    total = 0
    n = len(G)-1
    prob_1 = Decimal(0.5)
    for i in range(2**n):
        num = bin(i)[2:]
        num = "0"*(n-len(num)) + num
        permutation = [int(digit) for digit in list(num)]
        p0 = bn.permutation_probability(0, permutation)
        p1 = bn.permutation_probability(1, permutation)
        if ebm.predict(np.asarray(permutation))[0] == 0:
            total += p0 * (1-prob_1)
        else:
            total += p1 * prob_1
    print(total)
    predictions = ebm.predict(X_test)
    print(np.sum(predictions==y_test)/len(y_test))
    print()
    
    # Generate GAM polynomial in LaTeX format
    p = ebm_polynomial(ebm)
    linear = [f"{p[k]:.2f}{k}" for k in p if "*" not in k]
    const = linear[-1]
    del linear[-1]
    linear.insert(0, const) # Move constant to front
    interaction = [f"{p[k]:.2f}{k}" for k in p if "*" in k]
    for i, term in enumerate(linear): # Put curly braces for subscripts
        if "_" in term:
            linear[i] = term.replace("_", "_{") + "}"
    for i, term in enumerate(interaction): # Put curly braces for subscripts
        term = term.replace("*","").split("x_")
        term = term[0] + "x_{" + term[1] + "}x_{" + term[2] + "}"
        interaction[i] = term
    out_str = " + ".join(linear).replace("const", "")
    out_str += " + "
    out_str += " + ".join(interaction).replace("*", "")
    out_str = out_str.replace("+ -", "- ")
    latex_polynomials_gam.append(out_str)

Seed:   0%|          | 0/10 [00:00<?, ?it/s]

0.9960541388970632879614570547
0.995361328125

0.9991508868257527486920530946
0.999267578125

0.9987256478048793455196932591
0.99951171875

0.9963562011595956743700265886
0.99609375

0.9985726675602842780746943549
0.998779296875

0.9948303112887125445053998616
0.995849609375

0.9994226695664923902020569872
0.999755859375

0.9934054226008884172707543779
0.994140625

0.9976892721286398309529534925
0.997802734375

0.9982719631343544224040461023
0.998779296875



In [31]:
y_test = np.asarray(np.random.choice((0,1), 100_000))
X_test = bn.generate_dataset(y_test, seed=123)

predictions = ebm.predict(X_test)
np.sum(predictions==y_test)/len(y_test)

0.99829

In [78]:
# total_p0 = Decimal(0)
# total_p1 = Decimal(0)
# for i in range(2**n):
#     num = bin(i)[2:]
#     num = "0"*(n-len(num)) + num
#     permutation = [int(digit) for digit in list(num)]
#     p0 = bn.permutation_probability(0, permutation)
#     p1 = bn.permutation_probability(1, permutation)
#     if ebm.predict(np.asarray(permutation))[0] == 0:
#         total_p0 += p0
#     else:
#         total_p1 += p1
    

total = 0
n = len(G)-1
prob_1 = Decimal(0.5)
for i in range(2**n):
    num = bin(i)[2:]
    num = "0"*(n-len(num)) + num
    permutation = [int(digit) for digit in list(num)]
    p0 = bn.permutation_probability(0, permutation)
    p1 = bn.permutation_probability(1, permutation)
    if ebm.predict(np.asarray(permutation))[0] == 0:
        total += p0 / total_p0
    else:
        total += p1 / total_p1
print(total)

0.06166054283628693640709811815


In [77]:
total_p0 = Decimal(0)
total_p1 = Decimal(0)
for i in range(2**n):
    num = bin(i)[2:]
    num = "0"*(n-len(num)) + num
    permutation = [int(digit) for digit in list(num)]
    p0 = bn.permutation_probability(0, permutation)
    p1 = bn.permutation_probability(1, permutation)
    total_p0 += p0
    total_p1 += p1

total = 0
n = len(G)-1
prob_1 = Decimal(0.5)
for i in range(2**n):
    num = bin(i)[2:]
    num = "0"*(n-len(num)) + num
    permutation = [int(digit) for digit in list(num)]
    p0 = bn.permutation_probability(0, permutation)
    p1 = bn.permutation_probability(1, permutation)
    if ebm.predict(np.asarray(permutation))[0] == 0:
        total += p0 / total_p0 * prob_1 / (p0+p1)
    else:
        total += p1 / total_p1 * prob_1 / (p0+p1)
print(total)

2041.885143751709106421837398


In [75]:
total = 0
n = len(G)-1
prob_1 = Decimal(0.5)
for i in range(2**n):
    num = bin(i)[2:]
    num = "0"*(n-len(num)) + num
    permutation = [int(digit) for digit in list(num)]
    p0 = bn.permutation_probability(0, permutation)
    p1 = bn.permutation_probability(1, permutation)
    if ebm.predict(np.asarray(permutation))[0] == 0:
        total += p0 * prob_1 / (p0+p1)
    else:
        total += p1 * prob_1 / (p0+p1)
print(total)

0.9383394571637130635929018818


In [37]:
from sklearn.metrics import r2_score
r2_score(predictions, y_test)

0.993164060870185

In [42]:
X_train.corr()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
1,1.000000,-0.923503,-0.020870,-0.097970,0.723090,-0.125865,0.033465,0.720049,0.224539,0.724314,0.347037,0.422008,-0.168176,0.158426
2,-0.923503,1.000000,0.022784,0.068251,-0.746227,0.126362,-0.033274,-0.714032,-0.224440,-0.719772,-0.342853,-0.416960,0.168249,-0.158256
3,-0.020870,0.022784,1.000000,0.069079,0.025212,0.020891,0.044765,0.019630,-0.004880,0.018047,0.006498,0.030519,-0.013373,0.019344
4,-0.097970,0.068251,0.069079,1.000000,0.039173,0.006962,0.009356,-0.027820,-0.005629,-0.019750,-0.011045,-0.027733,0.014832,-0.013468
5,0.723090,-0.746227,0.025212,0.039173,1.000000,-0.116907,0.035353,0.586939,0.182279,0.589660,0.278759,0.346930,-0.145805,0.134463
6,-0.125865,0.126362,0.020891,0.006962,-0.116907,1.000000,-0.003569,-0.105779,-0.030266,-0.111265,-0.045543,-0.043945,0.017750,-0.009309
7,0.033465,-0.033274,0.044765,0.009356,0.035353,-0.003569,1.000000,0.038344,-0.004072,0.034575,0.016583,0.027285,-0.004681,0.015519
8,0.720049,-0.714032,0.019630,-0.027820,0.586939,-0.105779,0.038344,1.000000,-0.113017,0.921833,0.092297,0.542362,0.068233,-0.130368
9,0.224539,-0.224440,-0.004880,-0.005629,0.182279,-0.030266,-0.004072,-0.113017,1.000000,-0.046894,0.478984,-0.287145,-0.258128,0.291267
10,0.724314,-0.719772,0.018047,-0.019750,0.589660,-0.111265,0.034575,0.921833,-0.046894,1.000000,0.132979,0.503026,0.129512,-0.207810


In [42]:
for poly1, poly2 in zip(latex_polynomials_bn, latex_polynomials_gam):
#     print("$$"+poly1+"$$")
#     print()
    print("$$"+poly2+"$$")
    print()
    print("---")
    print()5.29x_{8}x_{9}

$$9.00 + 6.80x_{1} + 0.83x_{2} + 2.26x_{3} + 3.45x_{4} - 2.20x_{5} + 2.43x_{6} - 3.53x_{7} - 11.10x_{8} - 2.04x_{9} - 2.91x_{10} - 4.59x_{11} + 1.28x_{12} - 3.53x_{13} - 3.43x_{14} - 2.84x_{1}x_{3} + 0.87x_{1}x_{6} + 3.12x_{2}x_{3} - 0.18x_{2}x_{6} - 0.13x_{3}x_{4} - 5.12x_{3}x_{6} + 1.55x_{3}x_{7} + 1.02x_{6}x_{7} + 5.29x_{8}x_{9} + 3.83x_{8}x_{11} - 2.74x_{9}x_{11} + 5.54x_{10}x_{13}$$

---

$$13.89 - 1.81x_{1} + 3.05x_{2} + 0.39x_{3} - 1.42x_{4} - 2.28x_{5} - 6.27x_{6} + 0.67x_{7} - 5.93x_{8} + 0.10x_{9} - 8.61x_{10} - 3.19x_{11} - 7.49x_{12} - 2.16x_{13} + 0.12x_{14} + 2.73x_{1}x_{4} - 1.85x_{1}x_{5} - 3.83x_{2}x_{4} + 1.99x_{2}x_{5} + 1.94x_{3}x_{4} - 1.55x_{3}x_{5} + 4.04x_{3}x_{6} - 3.89x_{8}x_{9} + 5.84x_{8}x_{10} + 4.65x_{9}x_{11} + 4.70x_{10}x_{13} + 2.91x_{11}x_{12}$$

---

$$-7.78 + 3.28x_{1} + 8.78x_{2} + 7.00x_{3} + 2.00x_{4} + 0.59x_{5} - 3.45x_{6} + 2.24x_{7} + 3.49x_{8} + 1.71x_{9} - 5.47x_{10} + 5.13x_{11} + 1.24x_{12} - 2.20x_{13} - 3.87x_{14} - 4.99x_{1}x_{3} - 0.32